In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from hlsstack.hls_funcs.smooth import despike_ts, double_savgol
from hlsstack.hls_funcs.masks import bolton_mask_np

In [ ]:
df_ts = pd.read_csv('../data/raw_tmp/ts_raw_vor_plot.csv', parse_dates=[1])

In [ ]:
df_ts['duplicated'] = df_ts.duplicated(subset=['Id', 'Date'], keep=False)
df_ts['dup_first'] = df_ts.duplicated(subset=['Id', 'Date'], keep='first')
df_ts['dup_last'] = df_ts.duplicated(subset=['Id', 'Date'], keep='last')

In [ ]:
df_ts = df_ts[
(~df_ts['duplicated']) |
(df_ts['Date'].dt.month < 3) & df_ts['duplicated'] & (~df_ts['dup_last']) |
(df_ts['Date'].dt.month > 10) & df_ts['duplicated'] & (~df_ts['dup_first'])]

In [ ]:
test.duplicated(subset=['Id', 'Date']).any()

In [ ]:
cols = [x for x in df_ts.columns[2:] if '_smooth' not in x]

In [ ]:
# plot smoothed data for a single plot
plot = '20SE_P3'
fig, axs = plt.subplots(figsize=(20, 3*len(cols)), nrows=len(cols))
for idx, ax in enumerate(axs):
    df_ts[df_ts['Id'] == plot].plot.scatter(x='Date', y=cols[idx], ax=ax, c='black', s=10, alpha=0.5)
    df_ts[df_ts['Id'] == plot].plot(x='Date', y=cols[idx] + '_smooth', ax=ax, c='red')

In [ ]:
df_ts['NDVI_dv1'] = df_ts.groupby([df_ts['Date'].dt.year, 'Id'])['NDVI_smooth'].transform(lambda x: x.diff())

In [ ]:
# plot updated smooth with start of season (blue) and peak of season (red)
plot = '20SE_P3'
var1 = 'NDVI'
var2 = 'NDVI_dv1'

fig, axs = plt.subplots(figsize=(20, 6), nrows=2)
df_ts[df_ts['Id'] == plot].plot.scatter(x='Date', y=var1, ax=axs[0], c='black', s=10, alpha=0.5)
df_ts[df_ts['Id'] == plot].plot(x='Date', y= var1 + '_smooth', ax=axs[0], c='red')

df_ts[df_ts['Id'] == plot].plot(x='Date', y= var2 , ax=axs[1], c='red')


In [ ]:
# plot updated smooth with start of season (blue) and peak of season (red)
plot = '20SE_P3'
var1 = 'SATVI'
var2 = 'MTVI1'

fig, axs = plt.subplots(figsize=(20, 6), nrows=2)
df_ts[df_ts['Id'] == plot].plot.scatter(x='Date', y=var1, ax=axs[0], c='blue', s=10)
df_ts[df_ts['Id'] == plot].plot(x='Date', y= var1 + '_smooth', ax=axs[0], c='blue')

df_ts[df_ts['Id'] == plot].plot.scatter(x='Date', y=var2, ax=axs[1], c='red', s=10)
df_ts[df_ts['Id'] == plot].plot(x='Date', y= var2 + '_smooth', ax=axs[1], c='red')

for ax in axs:
    ax.axvline(pd.Timestamp('2016-06-12'), color='g')
    ax.axvline(pd.Timestamp('2016-06-30'), color='r')
    ax.axvline(pd.Timestamp('2016-10-5'), color='g')

In [ ]:
def bolton_mask(blue, swir2):

    indices = blue.index

    ts_blue = blue.values
    ts_swir2 = swir2.values
    
    def cloud_outlier_mask(da_blue):
        blue_ts = da_blue / 10000.0
        cloud_mask = np.zeros_like(blue_ts)
        for idx in range(len(blue_ts)):
            if not np.isnan(blue_ts[idx]):
                idx_clear = np.where(~np.isnan(blue_ts))[0]
                if idx == np.min(idx_clear):
                    continue
                else:
                    idx_pre = np.max(idx_clear[idx_clear < idx])
                    blue_diff = blue_ts[idx] - blue_ts[idx_pre]
                    cloud_thresh = 0.03 * (1 + (idx - idx_pre) / 30)
                    if blue_diff > cloud_thresh:
                        blue_ts[idx] = np.nan
                        cloud_mask[idx] = 1
                    else:
                        continue
            else:
                continue
        return cloud_mask

    def shadow_outlier_mask(da_swir2):
        swir2_ts = da_swir2.copy()
        shadow_mask = np.zeros_like(swir2_ts)
        for idx in range(len(swir2_ts)):
            if not np.isnan(swir2_ts[idx]):
                idx_clear = np.where(~np.isnan(swir2_ts))[0]
                if idx == np.min(idx_clear):
                    continue
                elif idx == np.max(idx_clear):
                    try:
                        idx_pre = idx_clear[idx_clear < idx][-1]
                        idx_pre2 = idx_clear[idx_clear < idx][-2]
                        y = np.array([swir2_ts[idx_pre2], swir2_ts[idx_pre]])
                        x = np.array([idx_pre2, idx_pre])
                        dx = np.diff(x)
                        dy = np.diff(y)
                        slope = dy / dx
                        swir2_interp = swir2_ts[idx_pre] + slope[0] * (idx - idx_pre)
                        swir2_diff = swir2_interp - swir2_ts[idx]
                        if (swir2_ts[idx_pre] - swir2_ts[idx_pre2]) == 0:
                            shadow_val = np.nan
                        else:
                            shadow_val = swir2_diff / (swir2_ts[idx_pre] - swir2_ts[idx_pre2])
                        if (idx - idx_pre2 < 45) & (swir2_diff > 500) & (np.abs(shadow_val) > 2):
                            swir2_ts[idx] = np.nan
                            shadow_mask[idx] = 1
                        else:
                            continue
                    except IndexError:
                        continue
                else:
                    idx_pre = idx_clear[idx_clear < idx][-1]
                    idx_post = idx_clear[idx_clear > idx][0]
                    y = np.array([swir2_ts[idx_pre], swir2_ts[idx_post]])
                    x = np.array([idx_pre, idx_post])
                    dx = np.diff(x)
                    dy = np.diff(y)
                    slope = dy / dx
                    swir2_interp = swir2_ts[idx_pre] + slope[0] * (idx - idx_pre)
                    swir2_diff = swir2_interp - swir2_ts[idx]
                    if (swir2_ts[idx_post] - swir2_ts[idx_pre]) == 0:
                        shadow_val = np.nan
                    else:
                        shadow_val = swir2_diff / (swir2_ts[idx_post] - swir2_ts[idx_pre])
                    if (idx_post - idx_pre < 45) & (swir2_diff > 500) & (np.abs(shadow_val) > 2):
                        swir2_ts[idx] = np.nan
                        shadow_mask[idx] = 1
                    else:
                        continue
            else:
                continue
        return shadow_mask

    shadow_outliers = shadow_outlier_mask(ts_swir2)
    ts_blue[shadow_outliers == 1]= np.nan
    cloud_outliers = cloud_outlier_mask(ts_blue)
    mask = np.maximum(cloud_outliers, shadow_outliers)
    mask = pd.Series(mask, index=indices)
    return mask

In [ ]:
def adapt_smooth(ts, dates, despike=True, dat_thresh=None):
    ct_valid = sum(~np.isnan(ts[(dates.dt.month >= 3) & (dates.dt.month <= 10)]))
    if 180 / ct_valid > 15:
        despike = False
    if despike:
        if dat_thresh is None:
            _dat_thresh = np.ptp(ts.values) * 0.10
        else:
            _dat_thresh = dat_thresh
        ts_ds = despike_ts(ts.values, dat_thresh=_dat_thresh, days_thresh=45)
    else:
        ts_ds = ts.values
    if 180 / ct_valid > 10:
        ts_smooth = double_savgol(ts_ds, double=True, window1_max=7, window2=31, limit=91)
    elif 180 / ct_valid > 7:
         ts_smooth = double_savgol(ts_ds, double=True, window1_max=5, window2=41, limit=91)
    elif 180 / ct_valid > 5:
         ts_smooth = double_savgol(ts_ds, double=True, window1_max=5, window2=51, limit=91)
    else:
        ts_smooth = double_savgol(ts_ds, double=False, window2=51, limit=91)
    return pd.Series(ts_smooth, ts.index.get_level_values(-1))

def set_smooth(ts, dates, despike=True, dat_thresh=None):
    if despike:
        if dat_thresh is None:
            _dat_thresh = np.ptp(ts.values[~np.isnan(ts.values)]) * 0.10
        else:
            _dat_thresh = dat_thresh
        ts_ds = despike_ts(ts.values, dat_thresh=_dat_thresh, days_thresh=45)
    ts_smooth = double_savgol(ts_ds, double=True, window1_max=9, window2=41, limit=91)
    return pd.Series(ts_smooth, ts.index.get_level_values(-1))

# dictionary specifying functions for each vegetation index to calculate and extract
veg_list = ['NDVI',
    'DFI',
    'NDTI',
    'SATVI',
    'NDII7',
    'BAI_126',
    'BAI_136',
    'BAI_146',
    'BAI_236',
    'BAI_246',
    'BAI_346']

# dictionary specifying individual bands to extract
band_list = ['BLUE', 'GREEN', 'RED', 'NIR1', 'SWIR1', 'SWIR2']

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
ps_bolton = df_ts.groupby('Id').progress_apply(lambda x: bolton_mask_np(x['BLUE'], x['SWIR2']))

In [ ]:
df_ts.loc[df_ts[ps_bolton.droplevel(0) == 1.0].index, veg_list + band_list] = np.nan

In [ ]:
despike_dict = {
    'NDVI': 0.05,
    'DFI': 2.0,
    'NDTI': None
}

In [ ]:
# smooth all vegetation indices to gapfill
for vegidx in veg_list:
    #df_yr_ts[vegidx + '_smooth'] = df_yr_ts.groupby('Id')[vegidx].transform(lambda x: double_savgol(x.values))
    vals_smooth = df_ts.groupby('Id').progress_apply(lambda x: set_smooth(x[vegidx], pd.to_datetime(x['Date'])))
    df_ts[vegidx + '_smooth2'] = vals_smooth.droplevel(list(np.arange(vals_smooth.index.nlevels-1)))
for band in band_list:
    #df_yr_ts[band + '_smooth'] = df_yr_ts.groupby('Id')[band].transform(lambda x: double_savgol(x.values))
    vals_smooth = df_ts.groupby('Id').progress_apply(lambda x: set_smooth(x[band], pd.to_datetime(x['Date'])))
    df_ts[band + '_smooth2'] = vals_smooth.droplevel(list(np.arange(vals_smooth.index.nlevels-1)))

In [ ]:
# plot updated smooth with start of season (blue) and peak of season (red)
plot = '10S_P1'
var1 = 'DFI'
var2 = 'NDVI'

fig, axs = plt.subplots(figsize=(20, 6), nrows=2)
df_ts[df_ts['Id'] == plot].plot.scatter(x='Date', y=var1, ax=axs[0], c='blue', s=15)
df_ts[df_ts['Id'] == plot].plot(x='Date', y= var1 + '_smooth', ax=axs[0], c='blue')
df_ts[df_ts['Id'] == plot].plot(x='Date', y= var1 + '_smooth2', ax=axs[0], c='blue', linestyle='dashed')


df_ts[df_ts['Id'] == plot].plot.scatter(x='Date', y=var2, ax=axs[1], c='red', s=10)
df_ts[df_ts['Id'] == plot].plot(x='Date', y= var2 + '_smooth', ax=axs[1], c='red')
df_ts[df_ts['Id'] == plot].plot(x='Date', y= var2 + '_smooth2', ax=axs[1], c='red', linestyle='dashed')

In [ ]:
# plot updated smooth with start of season (blue) and peak of season (red)
plot = '15E_P3'
var1 = 'NDTI'
var2 = 'NDVI'

fig, axs = plt.subplots(figsize=(20, 6), nrows=2)
df_ts[df_ts['Id'] == plot].plot.scatter(x='Date', y=var1, ax=axs[0], c='blue', s=5)
#df_ts[df_ts['Id'] == plot].plot(x='Date', y= var1 + '_smooth', ax=axs[0], c='blue')
df_ts[df_ts['Id'] == plot].plot(x='Date', y= var1 + '_smooth2', ax=axs[0], c='blue', linestyle='solid')


df_ts[df_ts['Id'] == plot].plot.scatter(x='Date', y=var2, ax=axs[1], c='red', s=5)
#df_ts[df_ts['Id'] == plot].plot(x='Date', y= var2 + '_smooth', ax=axs[1], c='red')
df_ts[df_ts['Id'] == plot].plot(x='Date', y= var2 + '_smooth2', ax=axs[1], c='red', linestyle='solid')

In [ ]:
# plot updated smooth with start of season (blue) and peak of season (red)
plot = '10S_P3'
var1 = 'BAI_136'
var2 = 'NDVI'

fig, axs = plt.subplots(figsize=(20, 6), nrows=2)
df_ts[df_ts['Id'] == plot].plot.scatter(x='Date', y=var1, ax=axs[0], c='blue', s=5)
#df_ts[df_ts['Id'] == plot].plot(x='Date', y= var1 + '_smooth', ax=axs[0], c='blue')
df_ts[df_ts['Id'] == plot].plot(x='Date', y= var1 + '_smooth2', ax=axs[0], c='blue', linestyle='solid')


df_ts[df_ts['Id'] == plot].plot.scatter(x='Date', y=var2, ax=axs[1], c='red', s=5)
#df_ts[df_ts['Id'] == plot].plot(x='Date', y= var2 + '_smooth', ax=axs[1], c='red')
df_ts[df_ts['Id'] == plot].plot(x='Date', y= var2 + '_smooth2', ax=axs[1], c='red', linestyle='solid')

In [ ]:
import re
# rename smoothed columns and drop originals
df_ts_out = df_ts.drop(columns=veg_list + band_list + ['_'.join([x, 'smooth']) for x in veg_list + band_list])
col_rename_dict = {c: re.sub('_smooth2', '', c) for c in df_ts_out.columns if '_smooth2' in c}
df_ts_out = df_ts_out.rename(columns=col_rename_dict)

In [ ]:
import os
inDIR = '../data/training/'
inFILE = 'vor_2013_2022_cln_2023_08_29_plot_hls_idxs.csv'
inPATH = os.path.join(inDIR, inFILE)
df_vor = pd.read_csv(inPATH, parse_dates=[2, 3])

In [ ]:
df_vor_out = pd.merge(df_vor.drop(columns=veg_list + band_list), 
                     df_ts_out[['Id', 'Date'] + veg_list + band_list], 
                     on=['Id', 'Date'],
                     how='left')

In [ ]:
df_vor_out.to_csv(re.sub('.csv', '_test.csv', inPATH), index=False)